# Introduction

In [ ]:
import trimesh
from utils import *

In [ ]:
fname = 'meshes/plants-2.obj'
mesh = trimesh.load_mesh(fname)

In [ ]:
mesh.show()

# Example Plant Model

In [ ]:
from models import plant_v0

In [ ]:
print_python_source(plant_v0.run)

In [ ]:
mesh = plant_v0.run(mesh, 0, 28, 1)  # TODO: Animation of output?

In [ ]:
mesh.show()